In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
from pathlib import Path
from glob import glob
from time import time
import warnings

from utils import *
from model import CNN_m123 as CNN
from train_harcnn import *

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
plt.rcParams['figure.figsize'] = [30, 30]
np.random.seed(42)

LABEL = {'jog_forward': 0, 'jog_turn': 1, 'walk_forward': 2, 'walk_turn': 3, 'run_forward':4, 'stationary':5}
LABEL_INV = {0:'jog_forward', 1:'jog_turn', 2:'walk_forward', 3:'walk_turn', 4:'run_forward', 5:'stationary'}

n_sample = 100

# Possible Sampling Rates
#SR_GNSS = [1,2,3,4,5,6,7,8,9,10]
SR_GNSS = [1,2,4,10]
SR_IMU = [10,20,25,50,100]

# Sampling rate for each movement category in Hz
SR = {  
    LABEL['walk_forward']: [2,100],
    LABEL['walk_turn']: [2,100],
    LABEL['jog_forward']: [2,100],
    LABEL['jog_turn']: [2,100],
    LABEL['run_forward']: [2,100],
    LABEL['stationary']: [2,100],
}
# Power Consumption ([mA]*[V])
PW_GNSS = {
    0: 23,
    1: 35,
    2: 40,
    3: 44,
    4: 48,
    5: 50,
    6: 53,
    7: 57,
    8: 60,
    9: 63,
    10:65
    }

PW_IMU = {
    0:10,
    10:11,
    20:12,
    25:15,
    33:18,
    50:22,
    100:24,
}
dir_data = Path('./data/datasets_fin/')

images = np.load(dir_data/'train_images.npy')
labels = np.load(dir_data/'train_labels.npy')
gnss = np.load(dir_data/'train_gnss.npy')
images = images[np.newaxis, ...].reshape(-1,1,16,100)

train_images = {key: [] for key in LABEL}
train_gnss = {key: [] for key in LABEL}
for i in range(len(labels)):
    label = LABEL_INV[labels[i][0]]
    train_images[label].append(images[i])
    train_gnss[label].append(gnss[i])
    
    

images = np.load(dir_data/'test_images.npy')
labels = np.load(dir_data/'test_labels.npy')
gnss = np.load(dir_data/'test_gnss.npy')
images = images[np.newaxis, ...].reshape(-1,1,16,100)

test_images = {key: [] for key in LABEL}
test_gnss = {key: [] for key in LABEL}
for i in range(len(labels)):
    label = LABEL_INV[labels[i][0]]
    test_images[label].append(images[i])
    test_gnss[label].append(gnss[i])
    
for label in LABEL:
    print(f"{label}: train set : {len(train_images[label])} \t| testset: {len(test_images[label])}")


net = CNN(n_classes=6)
net.load_state_dict(torch.load(Path('./models/MyModel.pth')))

jog_forward: train set : 1919 	| testset: 100
jog_turn: train set : 887 	| testset: 100
walk_forward: train set : 2877 	| testset: 100
walk_turn: train set : 936 	| testset: 100
run_forward: train set : 929 	| testset: 100
stationary: train set : 396 	| testset: 100


<All keys matched successfully>

In [29]:
## Helper functions
def get_key(val, my_dict):
    return [keys for keys, values in my_dict.items() if val==values][0]


def show_gps_label(df, idx_start=0, idx_end=None, enu_format=True, annotate=False, markersize=10):
    time = df[['time']][idx_start:idx_end].to_numpy()
    lat  = df[['latitude']][idx_start:idx_end].to_numpy()
    lon  = df[['longitude']][idx_start:idx_end].to_numpy()
    enu_x = df[['enu_x']][idx_start:idx_end].to_numpy()
    enu_y = df[['enu_y']][idx_start:idx_end].to_numpy()
    label = df[['prediction']][idx_start:idx_end].to_numpy()
    
    pos_range = [129.3186, 129.3202, 36.0124, 36.0140] # POSTECH Field

    if(enu_format):
        lat = enu_y
        lon = enu_x
    else:
        #plt.axis(pos_range)
        img = plt.imread("background2.png")
        plt.imshow(img, extent=pos_range)
    print("DATAFRAME LENGTH: ",len(df))
    
    color = {'walk_forward': 'tab:green',
             'walk_turn': 'tab:blue',
             'jog_forward': 'tab:orange',
             'jog_turn': 'tab:purple',
             'run_forward': 'tab:red',
             'stationary': 'tab:magenta',
            }
    plt.title(color,fontsize=30)
    
    for i in range(len(df)):
        p = get_key(label[i])
        if p in LABEL:
            c = color[p]
        else:
            c = 'k'
        plt.plot(lon[i], lat[i], marker='o', color = c, markersize=markersize)
        
    if(annotate):
        for idx, xy in enumerate(zip(lon,lat)):
                if(idx % 10 == 0):
                    plt.annotate('{}'.format(df.index[idx]), xy=xy, textcoords='data', fontsize=20)
                    
def har_predict(images, sr_imu):
    images = np.array(images)
    with torch.no_grad():
        if(sr_imu != n_sample):
            images = downsample_imu(images, n_sample, sr_imu)
            images = interpolate_imu(images, sr_imu, n_sample)
        images = torch.Tensor(images)
        images.requires_grad = False
        net.eval()
        res = net(images)
        predictions = np.argmax(res.cpu().detach().numpy(), axis=-1)
    return predictions


def downsample_imu(images, from_f, to_f):
    interval = int(from_f/to_f)
    assert interval == float(from_f)/to_f, "The sampling rate ratio must be a integer"
    return images[:,:,:,::interval]


def interpolate_imu(a, from_f, to_f):
    interval = int(to_f/from_f)
    assert interval == float(to_f)/from_f, "The sampling rate ratio must be a integer"
    arr = np.zeros([a.shape[0], a.shape[1], a.shape[2], a.shape[3]*interval], dtype=a.dtype)
    arr[:,:,:,::interval] = a
    inc = np.zeros(a.shape)
    inc[:,:,:,:-1] = (a[:,:,:,1:] - a[:,:,:,:-1])/interval
    inc[:,:,:,-1] = inc[:,:,:,-2]
    for i in range(1,interval):
        arr[:,:,:,i::interval] = arr[:,:,:,::interval] + inc*i
    return arr

def get_accuracy(pred, label, confusion_matrix):
    correct = (pred == label).sum()
    for i in LABEL.values():
        confusion_matrix[label, i] = (pred == i).sum()
    return correct/len(pred)


def downsample(gnss_base, pred):
    n_samples = 100
    gnss_downsample = np.array(gnss_base).copy()
    
    gnss_cols = ['time','enu_x','enu_y','speed']
    
    f = lambda x: 10/SR[x][0]
    sample_interval = np.fromiter([f(x) for x in pred], dtype=float)
    
    for i, sample in enumerate(gnss_downsample):
        sample_points = np.floor(np.arange(10)*sample_interval[i]+0.5)
        sample_points = np.append(sample_points[sample_points<10], 10)
        drop_points = [x for x in np.arange(10) if x not in sample_points]
        sample[:, drop_points] = np.nan
    return gnss_downsample


def interpolate(gnss_downsample):
    gnss_interpolate = gnss_downsample[:,:-1,:-1].copy()
    gnss_cols = ['time','enu_x','enu_y','speed']
    for i, sample in enumerate(gnss_downsample):
        df = pd.DataFrame(sample[:-1].T, columns=gnss_cols)
        df = df.interpolate(method='polynomial', order=1, limit_direction='both')[:-1] # remove 11'th entry
        gnss_interpolate[i] = df.to_numpy().T
    return gnss_interpolate


def calc_error(gnss_true, gnss_interpolate):
    cols = ['time','enu_x','enu_y','speed']
    
    gnss_true = np.array(gnss_true)[:, :-1, :-1]
    
    x_org = gnss_true[cols.index('enu_x')]
    y_org = gnss_true[cols.index('enu_y')]
    x_est = gnss_interpolate[cols.index('enu_x')]
    y_est = gnss_interpolate[cols.index('enu_y')]
    
    dist_org = np.sqrt(np.square(x_org[1:]-x_org[:-1])+np.square(y_org[1:]-y_org[:-1]))
    dist_est = np.sqrt(np.square(x_est[1:]-x_est[:-1])+np.square(y_est[1:]-y_est[:-1]))
    
    dRMSE = np.sqrt(np.mean(np.square(dist_org-dist_est)))
    pRMSE = np.sqrt(np.mean(np.square(x_org-x_est)+np.square(y_org-y_est)))
    vRMSE = np.sqrt(np.mean(np.square(gnss_true[cols.index('speed')]-gnss_interpolate[cols.index('speed')])))
    
    return dRMSE, pRMSE, vRMSE


def calc_power(predictions):
    energy_consumption = {key: 0 for key in LABEL}
    motion_count = {key: 0 for key in LABEL}
    
    for label in LABEL.keys():
        motion_count[label] = (predictions == LABEL[label]).sum()
        
    for label in LABEL:
        energy_consumption[label] = motion_count[label] * PW_GNSS[SR[LABEL[label]][0]] * 0.1 * 0.001# Joule per second
        energy_consumption[label] += motion_count[label] * PW_IMU[SR[LABEL[label]][1]] * 0.1 * 0.001
        
    return np.mean(list(energy_consumption.values()))



In [8]:
# HAR Classification Test

confusion_matrix = np.zeros([6,6], dtype=int)

for i, label in enumerate(LABEL):
    images = test_images[label]
    gnss = test_gnss[label]
    for key in SR.keys():
        SR[key][1] = SR_IMU_PRE[key]
    predictions = har_predict(images, SR[LABEL[label]][1])
    har_acc[i] = get_accuracy(predictions, LABEL[label], confusion_matrix)
    print(label, SR[LABEL[label]][1], har_acc[i])
    
        
confusion_matrix

jog_forward 100 1.0
jog_turn 100 0.98
walk_forward 100 1.0
walk_turn 100 1.0
run_forward 100 1.0
stationary 100 0.99


array([[100,   0,   0,   0,   0,   0],
       [  0,  98,   0,   0,   2,   0],
       [  0,   0, 100,   0,   0,   0],
       [  0,   0,   0, 100,   0,   0],
       [  0,   0,   0,   0, 100,   0],
       [  0,   0,   1,   0,   0,  99]])

In [14]:
# First Step; GNSS

np.set_printoptions(precision=3)
SR_IMU_PRE = {  
    LABEL['walk_forward']: 100,
    LABEL['walk_turn']: 100,
    LABEL['jog_forward']: 100,
    LABEL['jog_turn']: 100,
    LABEL['run_forward']: 100,
    LABEL['stationary']: 100,
}

dfs_out = {key: [] for key in LABEL}
confusion_matrix = np.zeros([6,6], dtype=int)

# Calculate power, vRMSE, dRMSE
har_acc = np.zeros(len(LABEL))
power = np.zeros([len(LABEL), len(SR_GNSS)])
vRMSE = np.zeros([len(LABEL), len(SR_GNSS)])
dRMSE = np.zeros([len(LABEL), len(SR_GNSS)])
pRMSE = np.zeros([len(LABEL), len(SR_GNSS)])

for i, label in enumerate(LABEL):
    images = train_images[label]
    gnss = train_gnss[label]
    for key in SR.keys():
        SR[key][1] = SR_IMU_PRE[key]
    predictions = har_predict(images, SR[LABEL[label]][1])
    har_acc[i] = get_accuracy(predictions, LABEL[label], confusion_matrix)
    print(label, SR[LABEL[label]][0], har_acc[i])
    
    for j, sr_gnss in enumerate(SR_GNSS):
        for key in SR.keys():
            SR[key][0] = sr_gnss
        gnss_downsample = downsample(gnss, predictions)
        gnss_interpolate = interpolate(gnss_downsample)
        #print(sr_gnss, pd.DataFrame(gnss_interpolate[0].T,columns =  ['time','enu_x','enu_y','speed']))
        power[i,j] = np.mean(calc_power(predictions))
        dRMSE[i,j], pRMSE[i,j], vRMSE[i,j] = calc_error(gnss, gnss_interpolate)
        
confusion_matrix

jog_forward 100 1.0
jog_turn 100 0.992108229988726
walk_forward 100 0.9954814042405283
walk_turn 100 0.9925213675213675
run_forward 100 0.9989235737351991
stationary 100 0.9848484848484849


array([[1919,    0,    0,    0,    0,    0],
       [   0,  880,    0,    0,    7,    0],
       [   5,    0, 2864,    7,    1,    0],
       [   0,    0,    7,  929,    0,    0],
       [   1,    0,    0,    0,  928,    0],
       [   0,    0,    4,    2,    0,  390]])

In [33]:
# First step; IMU
SR_GNSS_PRE = {  
    LABEL['walk_forward']: 1,
    LABEL['walk_turn']: 10,
    LABEL['jog_forward']: 1,
    LABEL['jog_turn']: 2,
    LABEL['run_forward']: 2,
    LABEL['stationary']: 1,
}
confusion_matrix = np.zeros([len(SR_IMU),6,6], dtype=int)

# Calculate power, vRMSE, dRMSE
har_acc = np.zeros([len(LABEL), len(SR_IMU)])
power = np.zeros([len(LABEL), len(SR_GNSS)])
vRMSE = np.zeros([len(LABEL), len(SR_GNSS)])
dRMSE = np.zeros([len(LABEL), len(SR_GNSS)])
pRMSE = np.zeros([len(LABEL), len(SR_GNSS)])

for i, label in enumerate(LABEL):
    images = train_images[label]
    gnss = train_gnss[label]
    for j, sr_imu in enumerate(SR_IMU):
        for key in SR.keys():
            SR[key][0] = SR_GNSS_PRE[key]
            SR[key][1] = sr_imu
        predictions = har_predict(images, SR[LABEL[label]][1])
        har_acc[i,j] = get_accuracy(predictions, LABEL[label], confusion_matrix[j])
        print(label, SR[LABEL[label]][1], har_acc[i,j])

        for j, sr_gnss in enumerate(SR_GNSS):
            for key in SR.keys():
                SR[key][0] = sr_gnss
            gnss_downsample = downsample(gnss, predictions)
            gnss_interpolate = interpolate(gnss_downsample)
            #print(sr_gnss, pd.DataFrame(gnss_interpolate[0].T,columns =  ['time','enu_x','enu_y','speed']))
            power[i,j] = np.mean(calc_power(predictions))
            dRMSE[i,j], pRMSE[i,j], vRMSE[i,j] = calc_error(gnss, gnss_interpolate)
            
print(confusion_matrix)


jog_forward 10 0.9911412193850964
jog_forward 20 1.0
jog_forward 25 1.0
jog_forward 50 1.0
jog_forward 100 1.0
jog_turn 10 0.705749718151071
jog_turn 20 0.895152198421646
jog_turn 25 0.9560315670800451
jog_turn 50 0.9763246899661782
jog_turn 100 0.992108229988726
walk_forward 10 0.9690649982620786
walk_forward 20 0.9937434827945777
walk_forward 25 0.9954814042405283
walk_forward 50 0.9947862356621481
walk_forward 100 0.9954814042405283
walk_turn 10 0.9658119658119658
walk_turn 20 0.9903846153846154
walk_turn 25 0.9925213675213675
walk_turn 50 0.9935897435897436
walk_turn 100 0.9925213675213675
run_forward 10 0.93756727664155
run_forward 20 1.0
run_forward 25 1.0
run_forward 50 1.0
run_forward 100 0.9989235737351991
stationary 10 0.9848484848484849
stationary 20 0.9873737373737373
stationary 25 0.9848484848484849
stationary 50 0.9848484848484849
stationary 100 0.9848484848484849
[[[1902    1    4    2   10    0]
  [  18  626    0    0  243    0]
  [   6    1 2788   70    3    9]
  [   0

In [5]:
for i in range(power.shape[0]):
    #print(get_key(i, LABEL))
    for j in range(power.shape[1]):
        print(power[i,j])
    print()

print("===================================")
for i in range(vRMSE.shape[0]):
    #print(get_key(i, LABEL))
    for j in range(vRMSE.shape[1]):
        print(vRMSE[i,j])
    print()
    
print("===================================")
for i in range(pRMSE.shape[0]):
    #print(get_key(i, LABEL))
    for j in range(pRMSE.shape[1]):
        print(pRMSE[i,j])
    print()
    
print("===================================")
for i in range(dRMSE.shape[0]):
    #print(get_key(i, LABEL))
    for j in range(dRMSE.shape[1]):
        print(dRMSE[i,j])
    print()

1.8870166666666668
2.0469333333333335
2.3028000000000004
2.8465166666666666

0.8722166666666666
0.9461333333333334
1.0644
1.3157166666666666

2.82905
3.0687999999999995
3.452400000000001
4.26755

0.9203999999999999
0.9984000000000002
1.1232
1.3884

0.9135166666666668
0.9909333333333334
1.1148000000000002
1.3780166666666667

0.3894
0.42240000000000005
0.47520000000000007
0.5874

0.09811943026833496
0.11476600784748249
0.07866815187887052
0.0

0.19404385877970318
0.10001652947266791
0.07954517536583004
0.0

0.063366126015879
0.06278562973371227
0.04615088653365509
0.0

0.0399090130303915
0.036330053932339984
0.035078133417477245
0.0

0.17172816869506224
0.10746033324919589
0.08984498104491023
0.0

0.005619231585034596
0.005388516123193509
0.005186911208733544
0.0

0.23175607801347917
0.19274355979804053
0.20662569070985565
0.0

0.31148858849090727
0.18588190980297387
0.19300315175969354
0.0

0.07372571396686588
0.06576901127458916
0.05086594940195247
0.0

0.07092375962285023
0.0599473197

In [32]:
import multiprocessing as mp

def worker(vRMSE,pRMSE,power,beta,proc_id,return_dict):
    res = []
    return_dict[proc_id] = []
    #for alpha in np.linspace(0,1,num=10)
    #print(proc_id)
    #opt_gnss_sr = {key: 0 for key in LABEL}
    f = lambda x: SR_GNSS[x]
    for p, beta in enumerate(beta):
        if(proc_id == 0):
            print(p)
        for alpha in np.linspace(0.0,1.0,num=10000):
            G = alpha*power + (1-alpha)*((1-beta)*vRMSE+beta*dRMSE)
        #G = alpha*power + (1-alpha)*((1-beta)*vRMSE+beta*pRMSE)

            min_idx = np.argmin(G, axis=1)
            opt_gnss_sr = tuple(np.fromiter([f(x) for x in min_idx], dtype=int))

            if(opt_gnss_sr not in res):
                res.append(opt_gnss_sr)

         #   if(alpha >= 0.1):
               # if(opt_gnss_sr in [(1,10,1,1,2,1),(1,10,1,1,10,1),(1,10,1,2,10,1)]):
                #if(opt_gnss_sr in [(2,10,2,2,4,1)]):
               #     print("{}\t{:.5f}\t{:.5f}\t{}".format(proc_id, beta, alpha, list(opt_gnss_sr)))
 #   print(f"End {proc_id}")
    return_dict[proc_id] = res

n_procs = 50
betas = np.split(np.linspace(0.0,1.0,num=10000), n_procs)
procs = []
return_dict = mp.Manager().dict()
for index, beta in enumerate(betas):
    proc = mp.Process(target=worker, args=(vRMSE,pRMSE,power,beta,index,return_dict,))
    #proc.daemon = True
    procs.append(proc)
    proc.start()
        
for p in procs:
    p.join()
vals = return_dict.values()
keys = return_dict.keys()

#a = routine(vRMSE,pRMSE,power,np.linspace(0.99,1.1,num=10),1)

vals_new = []

for items in vals:
    for item in items:
        vals_new.append(item)
        
vals_new = set(vals_new)
vals_new

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


{(1, 1, 1, 1, 1, 1),
 (1, 2, 1, 1, 1, 1),
 (1, 2, 1, 1, 2, 1),
 (1, 2, 1, 1, 10, 1),
 (1, 2, 1, 2, 1, 1),
 (1, 2, 1, 2, 2, 1),
 (1, 10, 1, 1, 2, 1),
 (1, 10, 1, 1, 10, 1),
 (1, 10, 1, 2, 1, 1),
 (1, 10, 1, 2, 2, 1),
 (1, 10, 1, 2, 10, 1),
 (1, 10, 1, 10, 10, 1),
 (10, 10, 1, 1, 10, 1),
 (10, 10, 1, 2, 10, 1),
 (10, 10, 1, 10, 10, 1),
 (10, 10, 1, 10, 10, 10),
 (10, 10, 10, 10, 10, 1),
 (10, 10, 10, 10, 10, 10)}